<a href="https://colab.research.google.com/github/GayathriP4/Python/blob/main/mp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#done once
#open the url, grant permissions and then copy the authorization code provided 
#and paste in the box provided
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
#Setup 
#only done once
#all required packages are downloaded
!pip install -q wordcloud
import wordcloud

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
import os
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from nltk.stem import PorterStemmer as PS
from nltk.stem import WordNetLemmatizer as wnl
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from nltk.corpus import wordnet

In [ ]:
#opens the directory and returns path of all files

def lfld(path,fp):
    if os.path.isdir(path):
        l = os.listdir(path)
        for x in l:
            tp = path + '/' + x
            lfld(tp,fp)
    elif os.path.isfile(path):
        fp.append(path)
        return
    else: pass

In [ ]:
#reads a file
#calls preprocess function
def readf(n):
  f = open(n,'r',encoding='latin-1')
  i = f.read()
  f.close()
  return i

In [ ]:
s='back backing backers better' 
l = wnl()
tex = [ l.lemmatize(x,pos='a') for x in s.split() ]
print(tex)

p = PS()
tex = [ p.stem(x) for x in tex ]
t = [ " ".join(c for c in tex) ]

print(t)
  

['back', 'backing', 'backers', 'good']
['back back backer good']


In [ ]:
#preprocessing

def pp(i):
  #removing punctuation
  t = "".join(c for c in i if c not in string.punctuation)

  # converting to tokens
  t = word_tokenize(t)
    
  # removing numbers and converting to lowercase
  t = [ x.lower() for x in t if x.isalpha() ]
    
  #removing stopwords and words with length 1 or greater than 14
  sw = set(stopwords.words('english'))
  tex = [ x for x in t if x not in sw and len(x)>1 and len(x)<15 ]
    
  #lemmatization
  l = wnl()
  tex = [ l.lemmatize(x) for x in tex ]
  p
  #stemming
  p = PS()
  tex = [ p.stem(x) for x in tex ]
  t = [ " ".join(c for c in tex) ]
  
  #returning the processed string as a single element list
  return t 

In [ ]:
#tf-idf vectorisation
def tf_idf1(fp,n):
  vectorizer = TfidfVectorizer()
  response = vectorizer.fit_transform(fp)
  fname = vectorizer.get_feature_names()
  vc = response[0]

  d = {'word':fname,'tfidf':[]}
  for i in vc.T.toarray():
    d['tfidf'].append(i[0])
  
  D = pd.DataFrame(d)
  D = D.sort_values(by=["tfidf"],ascending=False)
  
  l = D.head(n)['word']
  s = []
  for i in l:
    s.append(i)

  l = D.head(n)['tfidf']
  s1 = []
  for i in l:
    #rounding the tf-idf score to 3 decimals
    t = round(i,3)
    s1.append(t)
  
  #returns the top 'n' words and their tf-idf scores
  return s,s1

In [ ]:
#retrieves filepaths of all files in the mentioned path
#preproceeses it and calculates tf-idf
def ind(path,n):
  fp = []
  
  print("\nFetching Files...")
  lfld(path,fp)
  print("\nFiles fetched from ",path,sep='\n\n',end='\n\n')
  
  tfidf = { 'doc' : [] }
  
  #creates dictionary for words and their tf-idf score
  for i in range(n):
    tfidf[i] = []
  for i in range(n):
    tfidf[i+n] = []
  
  #
  for f in fp:
    s = readf(f)
    s = pp(s)
    tfidf['doc'].append(f)
    tw,ts = tf_idf1(s,n)
    for i in range(10):
      tfidf[i].append(tw[i])
      tfidf[i+10].append(ts[i])
  print("Preprocessing completed and tf-idf calculated.")
  return tfidf

In [ ]:
def tiv():
  try:
    path = input("Enter your path : ")
    os.path.listdir(path)
  except: 
      path = '/content/gdrive/My Drive/np/train/sci.crypt'

  try:
    n = int(input("Enter your index degree (default 10) : "))
    if n<10: n = 10
  except:
    n = 10

  print("\n\npath chosen :",path)
  print("Index degree :",n)


  tfidf = ind(path,n)

  #getting doc name and words in 'tfidf' as a DataFrame
  df = pd.DataFrame(tfidf)
  print('\nDataFrame created.\n')

  return df,n

In [ ]:
#to replace punctuation with spaces in query
def replacer(t):
  for p in string.punctuation:
    t = t.replace(p,' ')
  return t

In [ ]:
#to remove duplicates in query
def remdupl(t):
  l=[]
  for w in t.split(' '):
    if w not in l: 
      l.append(w)
  return l

In [ ]:
#finds synonyms for words in query(if any)
def syn(w):
  try:
    s = wordnet.synsets(w)[0]
  except:
    return [w]
  l = s.lemmas()
  x = []
  for i in range(len(l)):
    x.append(l[i].name())
  return x
 

In [ ]:
#checks for the queried word
def check(q,df,n):
  x = []
  sc = []
  for i in range(n):
    it = str(i)
    l = df[df[it] == q]
    for j in l['doc']:
      x.append(j)
    for j in l[str(i+n)]:
      sc.append(j)
  return x,sc

In [ ]:
#adds the a doc and its score
def add(fl,l,score):
  for x in range(len(l)):
    i = l[x]
    j = score[x]
    if i in fl:
      fl[i]['c'] = fl[i]['c'] + 1
      fl[i]['s'] = fl[i]['s'] + j
    else:
      fl[i] = { 'c' : 1 , 's' : j}

In [ ]:
#creates a DataFrame and sorts it accordingly
def sq(fl):
  d = {'doc':[],'freq':[],'score':[]}
  for x in fl:
    d['doc'].append(x)
    d['freq'].append(fl[x]['c'])
    d['score'].append(fl[x]['s'])
  FL = pd.DataFrame(d)
  FL = FL.sort_values(by=['freq','score'],ascending=[False,False])
  l = []
  for it in FL['doc']:
    l.append(it)
  return l

In [ ]:
def ppq(df,a):
  if not(a):
    df,n = tiv()
  else:
    try:
      n = int(input("Enter your index degree (default 10) : "))
      if n<10: n = 10
    except:
      n = 10
  qf=[]
  g=''
  q = input("Enter query/filepath :\n")
  if os.path.isfile(q):
    q = readf(q)
    
  #replacing punctuation
  q = replacer(q)
    
  #preprocessing query  
  q = pp(q)[0]
    
  #removing duplicates
  q = remdupl(q)

  #finding synonyms(if any)
  for i in q:
    for s in syn(i):
      #qf.append(s)
      g = g+s+' '

  #preprocessing and duplicates again
  qfg = pp(g)
  qf = remdupl(qfg[0])
  return qf,df,n

In [ ]:
def qfind(qf,df,n):
  fl = {}
  for word in qf:
    l,score = check(word,df,n)
    add(fl,l,score)
  sl = sq(fl)
  return sl

In [ ]:
def qagg(sl):
  l = len(sl)
  i = 0
  mi = l//10
  ch = a = '<'
  b = '>'
  while ch in [a,b]:
    if ch == '<':
      if i!=0:
        i-=1
    else:
      if i!=mi:i+=1
    print('\nPage ',i+1,'of',mi+1,end='\n\n')
    si = i*10
    ei = si+10
    if ei>l:
      ei = l
    for x in sl[si:ei]:print(x)
    if i!=0:
      print('<',end=' ')
      a='<'
    else:
      a = None
    if i!=mi:
      print('>')
      b='>'
    else:
      b=None
    ch = input()
  else:
    print("Thank You!")

To index a document

In [ ]:
#To index a document specifically
#fp = '/content/gdrive/My Drive/np/train/sci.crypt/14147'
try:
  fp = input("Enter path : ")
  f = open(fp,'r')
except:
  fp = '/content/gdrive/My Drive/np/train/sci.crypt/14147'

try:
  n = int(input("Enter your index degree (default 10) : "))
  if n<10: n = 10
except:
  n = 10
print("\n\nFile chosen :",fp,end='\n\n')
#reading the file path and preprocesses the content
fp = readf(fp)
fp = pp(fp)

#tf-idf vectorisation is done
x,y = tf_idf1(fp,n)

#prints the corresponding indexes
print('The indices are : ')
for i in x :print(i)

Enter path : 
Enter your index degree (default 10) : 8


File chosen : /content/gdrive/My Drive/np/train/sci.crypt/14147

The indices are : 
ripem
key
rsa
pgp
use
public
messag
de
encrypt
mail


To index a list of documents in a folder/directory

In [ ]:
print("To index a list of documents in a folder/directory")

df,n = tiv()

#saving 'tfidf' to csv file
try:
  cp = input("Enter csv file path : ")
  df.to_csv(cp, index = False )
except:
  df.to_csv(r'/content/gdrive/My Drive/np/i.csv' , index = False )

print('Added content to csv file.')
try:
  print(cp)
except: 
  print('\n/content/gdrive/My Drive/np/i.csv')

To index a list of documents in a folder/directory
Enter your path : 
Enter your index degree (default 10) : 9


path chosen : /content/gdrive/My Drive/np/train/sci.crypt
Index degree : 10
Fetching Files...
Files fetched from  /content/gdrive/My Drive/np/train/sci.crypt

Preprocessing completed and tf-idf calculated.

DataFrame created.

Enter csv file path : i.csv
Added content to csv file.
i.csv


To query from a list of documents in folder/directory

In [ ]:
print("For query :\n")
cp = input("Enter your csv file :\n")
try:
  df = pd.read_csv(cp)
  qf,df,n = ppq(df,True)
except:
  qf,df,n = ppq([],False)
sl = qfind(qf,df,n)
qagg(sl)
#rob key without id?hood

For query :

Enter your csv file :
i.csv
Enter your index degree (default 10) : 2
Enter query/filepath :
key chip id

Page  1 of 25

/content/gdrive/My Drive/np/train/sci.crypt/15620
/content/gdrive/My Drive/np/train/sci.crypt/15498
/content/gdrive/My Drive/np/train/sci.crypt/15495
/content/gdrive/My Drive/np/train/sci.crypt/15426
/content/gdrive/My Drive/np/train/sci.crypt/15442
/content/gdrive/My Drive/np/train/sci.crypt/15502
/content/gdrive/My Drive/np/train/sci.crypt/15700
/content/gdrive/My Drive/np/train/sci.crypt/15301
/content/gdrive/My Drive/np/train/sci.crypt/15670
/content/gdrive/My Drive/np/train/sci.crypt/15455
>
>

Page  2 of 25

/content/gdrive/My Drive/np/train/sci.crypt/15640
/content/gdrive/My Drive/np/train/sci.crypt/15375
/content/gdrive/My Drive/np/train/sci.crypt/15706
/content/gdrive/My Drive/np/train/sci.crypt/15658
/content/gdrive/My Drive/np/train/sci.crypt/15693
/content/gdrive/My Drive/np/train/sci.crypt/15374
/content/gdrive/My Drive/np/train/sci.crypt/155